# HR Dataset Investigation

In [1]:
import pandas as pd



## Predict the probability of an candidate looking for a new job
Möbius · 23 Submissions · a month to go

Task Details
This dataset designed to understand the factors that lead a person will work for the company(leaving current job) ,and the goal of this task is building model(s) that uses the current credentials,demographics,experience to predict the probability of a candidate looking for a new job or will work for the company.

The whole data divided to train and test . Also sample submission has been provided correspond to enrollee_ id of test set ( enrolle_ id | target)

Note:

The dataset is imbalanced so it might affect your result if you dont handle it
Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality so encoding methods and techniques will help to boost models performance
Missing imputation strategy might affect the results so it can be a part of your pipeline as well.
Expected Submission
Solution Notebook with following mandatory outputs:

Prediction: for each row in test data the predicted probability of candidate looking for a job, (probability of the class with the greater label).Format is given in sample_submission.csv:
enrollee_id : Unique ID for enrollee
target : probability of an enrollee looking for a job change
Model Evaluation: roc_ auc_ score of (answer,prediction probability of rows in test data)
*answer: test target values(download from : https://www.kaggle.com/arashnic/job-change-dataset-answer)
Model Interpretation: Interpret model(s) such a way that illustrate which features affect candidates decision
Evaluation
The evaluation metric is area under the ROC curve score.

You have better first evaluate your model(s) with your selected strategy (KFold CV, hold_ out,…) and then predict test (sample_submission) and finally compare the result for your own with test target values and find out your model performance on unseen data and your model fitness.

The pipeline you apply for getting result (EDA, preprocessing, feature engineering and selection , model building and evaluation strategy and so on, will be evaluated by your model performance on unseen data(aug_test.csv)

**Goals**  
- Predict the probability of a candidate will work for the company
- Interpret model(s) such a way that illustrate which features affect candidate decision


In [2]:
# read in Kaggle dataset, use enrollee_id as index
# note: Kaggle has provided seperate train and test datasets, do not need to split
HRtrain = pd.read_csv('aug_train.csv')
HRtest = pd.read_csv('aug_test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
HRtrain.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [4]:
HRtest.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


In [5]:
sample.head()

,enrollee_id,target
0,32403,0.5
1,9858,0.5
2,31806,0.5
3,27385,0.5
4,27724,0.5


In [6]:
HRtrain.shape

(19158, 14)

In [7]:
HRtest.shape

(2129, 13)

In [8]:
sample.shape

(2129, 2)

## Notes from Kaggle setup

- The final submission appears to be a list of enrollee_id where I need to predict a 0 or 1 for each id
- so given the information in the training dataset build a model then use that model to predict the target value for the enrollees in the test set
- given the almost 20K observations in the train dataset will split this into train, validate, test to build and evaluate model prior to using on provided test dataset


## Data definitions from Kaggle


enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

gender: Gender of candidate

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

training_hours: training hours completed

target: 0 – Not looking for job change, 1 – Looking for a job change

In [ ]:
# split data in HRtrain dataset


# Prepare

- check for null values
- determine if will need to impute any null values from train_df

In [ ]:
# checking for null values that will need to be addressed
HRtrain.isnull().sum()

In [ ]:
HRtrain.nunique()